In [1]:
import keras
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [7]:
model = Sequential()
model.add(Conv2D(32,(3,3),activation = 'relu',input_shape =(150,150,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32,(3,3),activation = 'relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32,(3,3),activation = 'relu'))
model.add(MaxPooling2D())
model.add(Flatten())          
model.add(Dense(100,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))


In [8]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics =['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True
)

In [3]:
test_datagen = ImageDataGenerator(
        rescale = 1./255
)

In [5]:
training_set = train_datagen.flow_from_directory(
        'train',
        target_size = (150,150),
        batch_size = 16,
        class_mode = 'binary'
)

Found 7457 images belonging to 2 classes.


In [6]:
test_set = test_datagen.flow_from_directory(
    'test',
    target_size = (150,150),
    batch_size = 16,
    class_mode = 'binary'
)

Found 96 images belonging to 2 classes.


In [ ]:
history = model.fit_generator(
    training_set,
    epoch = 20,
    validation_data = test_set,
    
)

In [ ]:
model.save('mask_detectionCnn.h5',history)

In [8]:
import numpy as np
import cv2 
from keras.models import load_model
from keras.preprocessing import image


In [10]:
mymodel =load_model('mask_detectionCnn.h5')

In [11]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

In [20]:
cap= cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,720)
while cap.isOpened():
    _,img = cap.read()
    face = face_cascade.detectMultiScale(img,scaleFactor =1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h,x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_img = image.load_img('temp.jpg',target_size = (150,150,3))
        test_img = image.img_to_array(test_img)
        test_img = np.expand_dims(test_img,axis =0)
        
        pred = mymodel.predict(test_img)[0][0]
        if pred ==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
            cv2.rectangle(img,(x,y-40),(x+w,y),(0,0,255),-1)
            cv2.putText(img,'NO MASK',(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.rectangle(img,(x,y-40),(x+w,y),(0,255,0),-1)
            cv2.putText(img,'MASK',(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    cv2.imshow("Image",img)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
        

In [17]:
cap.release()
cv2.destroyAllWindows()